In [3]:
import numpy as np
import warnings
warnings.filterwarnings("ignore")
X = {}
y = {}
X["train"] = np.load("train_feature_save.npy",allow_pickle=True)
y["train"] = np.load("train_label_save.npy").squeeze()
X["dev"] = np.load("test_feature_save.npy")
y["dev"] = np.load("test_label_save.npy").squeeze()
X_text_train = np.load("X_text_train.npy")
X_text_dev = np.load("X_text_dev.npy")
X["train"].shape, X["dev"].shape

((11678, 768), (1298, 768))

In [4]:
from sklearn.linear_model import LogisticRegression
model = LogisticRegression(penalty='l2', C=0.1)
model.fit(X["train"], y["train"])
model.score(X["dev"], y["dev"])

0.9714946070878274

In [5]:
w = np.concatenate((model.coef_, model.intercept_[None, :]), axis=1)
F_train = np.concatenate([X["train"], np.ones((X["train"].shape[0], 1))], axis=1) # Concatenating one to calculate the gradient with respect to intercept
F_dev = np.concatenate([X["dev"], np.ones((X["dev"].shape[0], 1))], axis=1)

error_train = model.predict_proba(X["train"])[:, 1] - y["train"]
error_dev = model.predict_proba(X["dev"])[:, 1] - y["dev"]

gradient_train = F_train * error_train[:, None]
gradient_dev = F_dev * error_dev[:, None]
gradient_train.shape, gradient_dev.shape

((11678, 769), (1298, 769))

In [6]:
#from scipy import sparse
probs = model.predict_proba(X["train"])[:, 1]
hessian = F_train.T @ np.diag(probs * (1 - probs)) @ F_train / X["train"].shape[0] + 10 * np.eye(F_train.shape[1]) / X["train"].shape[0]
inverse_hessian = np.linalg.inv(hessian)

In [7]:
eps = 1 / X["train"].shape[0]
delta_k = -eps * inverse_hessian @ gradient_train.T
pred = np.reshape(model.predict_proba(X["dev"])[:, 1], (model.predict_proba(X["dev"])[:, 1].shape[0], 1))
grad_f = F_dev * (pred * (1 - pred))
delta_pred = grad_f @ delta_k
delta_pred.shape

(1298, 11678)

In [8]:
ls

CSV_essay_bertlin_alg1.csv    essay_data/
CSV_essay_bertlin_alg1.ipynb  new_predictions.npy
HOPE2_essay/                  old_predictions.npy
Smallest_k.ipynb              test_feature_save.npy
Use_approK.ipynb              test_label_save.npy
X_text_dev.npy                train_feature_save.npy
X_text_train.npy              train_label_save.npy
appro_ks_IP.npy


In [9]:
app_k = np.array(np.load("appro_ks_IP.npy", allow_pickle=True).squeeze())
pred = np.load("old_predictions.npy", allow_pickle=True)
app_k_nonon=[]
for i in range(X["dev"].shape[0]):
    print(app_k[i])
    if app_k[i] == None:
        app_k_nonon.append(X["train"].shape[0])
    else:
        app_k_nonon.append(app_k[i])

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
119
None
146
None
None
None
None
1515
None
None
None
None
None
381
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
91
None
None
None
None
None
None
None
3
None
None
None
None
None
None
None
None
None
None
None
None
None
None
83
None
None
None
None
None
None
None
750
None
None
None
683
None
None
None
None
None
None
80
None
None
None
None
None
49
None
None
3211
2242
None
None
None
None
None
None
None
None
None
None
225
None
None
None
None
None
None
16
117
None
None
None
74
None
115
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
313
None
None
None
199
None
None
None
None
None
None
None
None
None
None
130
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
None
34
None
281
None
None
None
None
None
None

In [10]:
X["train"].shape[0]

11678

In [11]:
sort_dev_k = np.array(app_k_nonon, dtype=int).argsort()
sort_dev_k

array([1161,  253,  950, ...,  450,  457, 1297])

In [12]:
app_k_nonon[253]

1

In [13]:
header = ["test_movie_id", "test_movie_review_text", "old_prediction", "right_prediction", "k", "successful_flip"]
for i in range(X["train"].shape[0]):
    header.append("train_example_" + str(i+1))

In [14]:
new_predictions = np.load("new_predictions.npy", allow_pickle=True)

In [15]:
data = []
for test_movie_id in sort_dev_k:
    
    test_point = []
    # id
    test_point.append(test_movie_id)
    # test_movie_review_text
    test_point.append("label: " + str(y["dev"][test_movie_id]) + "  doc: " + str(X_text_dev[test_movie_id]))
    test_point.append(pred[test_movie_id].item())
    test_point.append(np.round(pred[test_movie_id].item()) == y["dev"][test_movie_id].item())
    # k
    k = app_k[test_movie_id]
    test_point.append(str(k))
    print("k", k)
    if k == None:
        test_point.append(None)
        data.append(test_point)
        continue
    k = int(k)
    test_point.append(np.round(new_predictions[test_movie_id]) != np.round(pred[test_movie_id].item()))
        
    
    if pred[test_movie_id] > 0.5:
        top_k_index = delta_pred[test_movie_id].argsort()[-k:]
    else:
        top_k_index = delta_pred[test_movie_id].argsort()[:k]
    
    # training examples
    for idx in top_k_index:
        test_point.append("id: " + str(idx) + "  label: " + str(y["train"][idx]) + "  doc: " + str(X_text_train[idx]))
    data.append(test_point)

k 1
k 1
k 1
k 1
k 3
k 5
k 5
k 11
k 13
k 15
k 15
k 15
k 16
k 20
k 21
k 23
k 25
k 25
k 32
k 32
k 34
k 35
k 43
k 45
k 45
k 48
k 49
k 49
k 49
k 56
k 58
k 58
k 60
k 64
k 64
k 67
k 72
k 74
k 74
k 74
k 75
k 80
k 83
k 88
k 89
k 91
k 99
k 107
k 114
k 115
k 116
k 117
k 119
k 120
k 124
k 125
k 130
k 132
k 133
k 133
k 143
k 146
k 147
k 148
k 158
k 175
k 187
k 195
k 199
k 203
k 205
k 218
k 225
k 254
k 255
k 262
k 281
k 297
k 313
k 313
k 332
k 345
k 373
k 379
k 381
k 388
k 403
k 463
k 512
k 652
k 655
k 683
k 750
k 781
k 824
k 884
k 926
k 933
k 1008
k 1115
k 1196
k 1515
k 1515
k 1565
k 1620
k 1676
k 1797
k 2138
k 2242
k 3177
k 3211
k 3789
k 3805
k 5266
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k None
k No

In [16]:
import csv

with open('CSV_SST_bertlin_alg1.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)


In [43]:
ls

CSV_SST_LR_alg1.csv    IP_change_log.npy      difference_k_appk.npy
CSV_SST_LR_alg1.ipynb  IP_flip_log.npy        exact_ks_IP.npy
Errors_k.ipynb         Plot.ipynb             get_order.ipynb
GC_change.npy          RIF_change.npy         need_reg?.ipynb
GC_flip.npy            RIF_flip.npy           new_pred_k.npy
GD_change.npy          Same.jpg               new_predictions.npy
GD_flip.npy            Smallest_k.ipynb       old_predictions.npy
IF_change_log.npy      Use_approK.ipynb       order_IF.npy
IF_flip_log.npy        appro_ks_IP.npy        pred.npy


In [44]:
pwd

'/Users/eciel/Desktop/y3s2_sum/orf/summer/privacy_methods/LR/IF/Logistic_SST/IF_lr_fix'